In [1]:
import os, sys
#sys.path.append('/Users/hhsieh/anaconda3/envs/astroconda/lib/python3.6/site-packages')
#sys.path.append('/opt/local/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages')
#print(sys.version)
from astropy.io import fits
from astropy.io.fits import getheader
from astropy.modeling import models
from astropy import nddata
from astropy import units as u
import astropy.units as u
import astropy.coordinates as coord
from astropy.time import Time, TimeDelta
from astropy.table import Table
from astropy.coordinates import SkyCoord,Angle
from astroquery.jplhorizons import Horizons
import ccdproc as cp
from ccdproc import CCDData, ImageFileCollection
import numpy as np
import math, glob, os, bz2, subprocess, sys
import cosmics_py3
import scipy.signal as signal
import scipy.ndimage as ndimage
import datetime


In [2]:
def compress_file_fpack(filename):
    fpack_command  = '/Users/hhsieh/Astro/tools/cfitsio/fpack'
    delete_command = '/bin/rm'
    fpack_cmd = [fpack_command,filename]
    process = subprocess.call(fpack_cmd)
    if process == 0:
        delete_cmd = [delete_command,filename]
        process = subprocess.call(delete_cmd)
    else:
        print('fpack command failed.')
    return None

def decompress_file_fpack(filename):
    funpack_command = '/Users/hhsieh/Astro/tools/cfitsio/funpack'
    delete_command  = '/bin/rm'
    funpack_cmd = [funpack_command,filename]
    process = subprocess.call(funpack_cmd)
    if process == 0:
        delete_cmd = [delete_command,filename]
        process = subprocess.call(delete_cmd)
    else:
        print('funpack command failed.')
    return None



In [3]:
def update_metadata(fits_file):
    with fits.open(fits_file) as hdulist:
        data = hdulist[0].data
        hdr = hdulist[0].header
        
    #'EPOCH','CD1_1','CD1_2','CD2_1','CD2_2','CHECKSUM'

    # Remove keywords
    if 'EPOCH'    in hdr: del hdr['EPOCH']
    if 'CD1_1'    in hdr: del hdr['CD1_1']
    if 'CD1_2'    in hdr: del hdr['CD1_2']
    if 'CD2_1'    in hdr: del hdr['CD2_1']
    if 'CD2_2'    in hdr: del hdr['CD2_2']
    if 'CHECKSUM' in hdr: del hdr['CHECKSUM']
    if 'DATASUM'  in hdr: del hdr['DATASUM']
        
    fits.writeto(fits_file,data,hdr,overwrite=True,checksum=True)
    
    return None


In [4]:
def update_calib_file_headers(filepath):
    print('{:s} - Updating headers of calibration files in {:s}...'.format(datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S'),filepath))
    os.chdir(filepath)
    for fits_file_fz in sorted(glob.glob('*.fz')):
        decompress_file_fpack(fits_file_fz)
        fits_file = fits_file_fz[:-3]
        update_metadata(fits_file)
        compress_file_fpack(fits_file)
        
    return None


In [5]:
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2016B-LP-11/GN_2016B/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2017A-LP-11/GN_2017A/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2017B-LP-11/GN_2017B/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2018A-LP-11/GN_2018A/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2018B-LP-11/GN_2018B/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2019A-LP-11/GN_2019A/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2016B-LP-11/GS_2016B/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2017A-LP-11/GS_2017A/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2017B-LP-11/GS_2017B/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2018A-LP-11/GS_2018A/calibration_data/')
update_calib_file_headers('/volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GS-2018B-LP-11/GS_2018B/calibration_data/')
print('Done.')

2022-01-11 16:23:32 - Updating headers of calibration files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2016B-LP-11/GN_2016B/calibration_data/...
2022-01-11 16:24:54 - Updating headers of calibration files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2017A-LP-11/GN_2017A/calibration_data/...
2022-01-11 16:25:26 - Updating headers of calibration files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2017B-LP-11/GN_2017B/calibration_data/...
2022-01-11 16:26:19 - Updating headers of calibration files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2018A-LP-11/GN_2018A/calibration_data/...
2022-01-11 16:26:36 - Updating headers of calibration files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2018B-LP-11/GN_2018B/calibration_data/...
2022-01-11 16:27:05 - Updating headers of calibration files in /volumes/Fantom12a/BackupData/gemini/data_LLP/gemini_data.GN-2019A-LP-11/GN_2019A/calibration_data/...
2022